In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import numpy as np
import nibabel as nib
# ... rest of your imports and code

import os
import numpy as np
import torch
import pydicom
from pathlib import Path
import pickle
from tqdm import tqdm
import torch.nn.functional as F
from scipy.ndimage import zoom
import nibabel as nib
import warnings
warnings.filterwarnings('ignore')

print(f"Using device: {'cuda' if torch.cuda.is_available() else 'cpu'}")
print(f"PyTorch version: {torch.__version__}")

from ct_clip.ct_clip import CTCLIP
print("CT-CLIP imported successfully!")

Using device: cuda
PyTorch version: 2.1.0+cu121
CT-CLIP imported successfully!


In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import pandas as pd
from tqdm import tqdm

root_dicom_dir = r"C:\Users\20203686\OneDrive - TU Eindhoven\TUe\Master\Year 2\MASTER PROJECT\TEST CT SCANS"
root_nifti_dir = r"C:\Users\20203686\OneDrive - TU Eindhoven\TUe\Master\Year 2\MASTER PROJECT\NIFTI CT SCANS"

patient_dirs = [d for d in os.listdir(root_dicom_dir) if os.path.isdir(os.path.join(root_dicom_dir, d))]

metadata_records = []

for patient_id in tqdm(patient_dirs, desc="Patients"):
    patient_dicom_path = os.path.join(root_dicom_dir, patient_id)
    patient_nifti_path = os.path.join(root_nifti_dir, patient_id)
    os.makedirs(patient_nifti_path, exist_ok=True)

    scan_dirs = [d for d in os.listdir(patient_dicom_path) if os.path.isdir(os.path.join(patient_dicom_path, d))]

    for scan_name in tqdm(scan_dirs, desc=f"Scans for patient {patient_id}", leave=False):
        scan_dicom_path = os.path.join(patient_dicom_path, scan_name)

        dcm_files = [os.path.join(scan_dicom_path, f) for f in os.listdir(scan_dicom_path) if f.endswith('.dcm')]

        slices = [pydicom.dcmread(f) for f in dcm_files]
        slices.sort(key=lambda x: int(getattr(x, 'InstanceNumber', 0)))

        volume = np.stack([s.pixel_array for s in slices])

        # Extract DICOM metadata from first slice
        first_slice = slices[0]
        slope = float(getattr(first_slice, 'RescaleSlope', 1.0))
        intercept = float(getattr(first_slice, 'RescaleIntercept', 0.0))

        pixel_spacing = first_slice.PixelSpacing  # usually [row_spacing, col_spacing]
        xy_spacing = float(pixel_spacing[0])  # assuming isotropic in-plane spacing
        z_spacing = float(getattr(first_slice, 'SliceThickness', 1.0))

        # Prepare affine transformation for NIfTI
        spacing = [xy_spacing, xy_spacing, z_spacing]
        affine = np.diag(spacing + [1])

        # Save NIfTI file
        nifti_filename = f"{scan_name}.nii.gz"
        nifti_path = os.path.join(patient_nifti_path, nifti_filename)

        nifti_img = nib.Nifti1Image(volume, affine=affine)
        nib.save(nifti_img, nifti_path)

        # Collect metadata
        metadata_records.append({
            'PatientID': patient_id,
            'ScanName': scan_name,
            'VolumeName': nifti_filename,
            'RescaleSlope': slope,
            'RescaleIntercept': intercept,
            'XYSpacing': xy_spacing,
            'ZSpacing': z_spacing
        })

# Save metadata to CSV
metadata_df = pd.DataFrame(metadata_records)
metadata_csv_path = os.path.join(r"C:\Users\20203686\OneDrive - TU Eindhoven\TUe\Master\Year 2\MASTER PROJECT", "metadata.csv")
metadata_df.to_csv(metadata_csv_path, index=False)

print(f"Metadata saved to {metadata_csv_path}")
print(f"Processed {len(metadata_records)} volumes.")


Patients: 100%|██████████| 2/2 [00:17<00:00,  8.82s/it]

Metadata saved to C:\Users\20203686\OneDrive - TU Eindhoven\TUe\Master\Year 2\MASTER PROJECT\metadata.csv
Processed 5 volumes.


In [11]:
import os
import nibabel as nib
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from multiprocessing import Pool
from tqdm import tqdm

base_root = r"C:\Users\20203686\OneDrive - TU Eindhoven\TUe\Master\Year 2\MASTER PROJECT"
root_nifti_dir = r"C:\Users\20203686\OneDrive - TU Eindhoven\TUe\Master\Year 2\MASTER PROJECT\NIFTI CT SCANS"
metadata_csv_path = r"C:\Users\20203686\Downloads\train_metadata.csv"

# Load metadata
df = pd.read_csv(metadata_csv_path)

def read_nii_files(directory):
    nii_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.nii.gz'):
                nii_files.append(os.path.join(root, file))
    return nii_files

def read_nii_data(file_path):
    try:
        nii_img = nib.load(file_path)
        nii_data = nii_img.get_fdata()
        return nii_data
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

def resize_array(array, current_spacing, target_spacing):
    # array shape = (1, 1, D, H, W)
    original_shape = array.shape[2:]  # D, H, W
    scaling_factors = [current_spacing[i] / target_spacing[i] for i in range(len(original_shape))]
    new_shape = [int(original_shape[i] * scaling_factors[i]) for i in range(len(original_shape))]
    resized_array = F.interpolate(array, size=new_shape, mode='trilinear', align_corners=False).cpu().numpy()
    return resized_array

def process_file(file_path):
    img_data = read_nii_data(file_path)
    if img_data is None:
        print(f"Read {file_path} unsuccessful. Passing")
        return

    file_name = os.path.basename(file_path)

    # Get metadata row matching VolumeName (filename)
    row = df[df['VolumeName'] == file_name]
    if row.empty:
        print(f"No metadata found for {file_name}, skipping")
        return

    slope = float(row["RescaleSlope"].iloc[0])
    intercept = float(row["RescaleIntercept"].iloc[0])
    xy_spacing = float(row["XYSpacing"].iloc[0][1:-1].split(",")[0])
    z_spacing = float(row["ZSpacing"].iloc[0])

    # Target spacings
    target_x_spacing = 0.75
    target_y_spacing = 0.75
    target_z_spacing = 1.5

    current_spacing = (z_spacing, xy_spacing, xy_spacing)
    target_spacing = (target_z_spacing, target_x_spacing, target_y_spacing)

    img_data = slope * img_data + intercept

    hu_min, hu_max = -1000, 1000
    img_data = np.clip(img_data, hu_min, hu_max)
    img_data = (img_data / 1000).astype(np.float32)

    tensor = torch.tensor(img_data).unsqueeze(0).unsqueeze(0)
    resized_array = resize_array(tensor, current_spacing, target_spacing)
    resized_array = resized_array[0, 0]

    patient_id = os.path.basename(os.path.dirname(file_path))

    # Create patient folder inside main save folder
    save_folder = os.path.join(base_root, "NIFTI PRE CT SCANS", patient_id)
    os.makedirs(save_folder, exist_ok=True)

    npz_name = file_name.replace('.nii.gz', '.npz')
    save_path = os.path.join(save_folder, npz_name)
    np.savez_compressed(save_path, resized_array)
    print(f"Saved preprocessed file at {save_path}")


# Get all NIfTI files
nifti_files = read_nii_files(root_nifti_dir)

for f in tqdm(nifti_files, desc="Preprocessing NIfTI files"):
    process_file(f)

Preprocessing NIfTI files:  17%|█▋        | 1/6 [00:00<00:00,  5.12it/s]

No metadata found for THX AX MIP 10 8.nii.gz, skipping


Preprocessing NIfTI files:  33%|███▎      | 2/6 [00:00<00:01,  2.34it/s]

No metadata found for THX BB COR 3 3.nii.gz, skipping


Preprocessing NIfTI files:  67%|██████▋   | 4/6 [00:01<00:00,  2.44it/s]

No metadata found for THX BB SAG 3 3.nii.gz, skipping
No metadata found for MIP 10 8 AXIAL.nii.gz, skipping


Preprocessing NIfTI files:  83%|████████▎ | 5/6 [00:02<00:00,  1.36it/s]

No metadata found for Thorax_lever 1.25_1.25.nii.gz, skipping


Preprocessing NIfTI files: 100%|██████████| 6/6 [00:26<00:00,  4.46s/it]

Saved preprocessed file at C:\Users\20203686\OneDrive - TU Eindhoven\TUe\Master\Year 2\MASTER PROJECT\NIFTI PRE CT SCANS\Downloaded\train_10019_a_1.npz


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import argparse
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from transformer_maskgit import CTViT
from ct_clip import CTCLIP

# Sigmoid function
def sigmoid(tensor):
    return 1 / (1 + torch.exp(-tensor))

# Classifier model
class ImageLatentsClassifier(nn.Module):
    def __init__(self, trained_model, latent_dim, num_classes, dropout_prob=0.3):
        super(ImageLatentsClassifier, self).__init__()
        self.trained_model = trained_model
        self.dropout = nn.Dropout(dropout_prob)
        self.relu = nn.ReLU()
        self.classifier = nn.Linear(latent_dim, num_classes)

    def forward(self, latents=False, *args, **kwargs):
        kwargs['return_latents'] = True
        _, image_latents = self.trained_model(*args, **kwargs)
        image_latents = self.relu(image_latents)
        if latents:
            return image_latents
        image_latents = self.dropout(image_latents)
        return self.classifier(image_latents)

# Custom dataset for inference
class CTReportDatasetinfer(Dataset):
    def __init__(self, data_folder, csv_file, labels_csv):
        """
        data_folder: Directory containing .npz files (e.g., 'NIFTI PRE CT SCANS')
        csv_file: Path to metadata.csv
        labels_csv: Path to CSV with pathology labels (PatientID, ScanName, and 18 pathology columns)
        """
        self.data_folder = data_folder
        self.metadata = pd.read_csv(csv_file)
        self.labels_df = pd.read_csv(labels_csv)
        self.npz_files = []
        for root, _, files in os.walk(data_folder):
            for file in files:
                if file.endswith('.npz'):
                    self.npz_files.append(os.path.join(root, file))
        self.pathologies = ['Medical material', 'Arterial wall calcification', 'Cardiomegaly', 'Pericardial effusion',
                            'Coronary artery wall calcification', 'Hiatal hernia', 'Lymphadenopathy', 'Emphysema',
                            'Atelectasis', 'Lung nodule', 'Lung opacity', 'Pulmonary fibrotic sequela',
                            'Pleural effusion', 'Mosaic attenuation pattern', 'Peribronchial thickening',
                            'Consolidation', 'Bronchiectasis', 'Interlobular septal thickening']

    def __len__(self):
        return len(self.npz_files)

    def __getitem__(self, idx):
        npz_path = self.npz_files[idx]
        patient_id = os.path.basename(os.path.dirname(npz_path))
        scan_name = os.path.basename(npz_path).replace('.npz', '')
        
        # Load .npz file
        data = np.load(npz_path)
        img_data = data['arr_0'].astype(np.float32)
        
        # Resize to (480, 480, 480)
        target_shape = (480, 480, 480)
        img_data = self.resize_volume(img_data, target_shape)
        
        # Convert to tensor and add channel dimension
        img_tensor = torch.tensor(img_data).unsqueeze(0)  # Shape: (1, D, H, W)
        
        # Get labels from labels_csv
        label_row = self.labels_df[(self.labels_df['PatientID'] == patient_id) & 
                                  (self.labels_df['ScanName'] == scan_name)]
        if label_row.empty:
            labels = torch.zeros(len(self.pathologies), dtype=torch.float32)
        else:
            labels = torch.tensor(label_row[self.pathologies].values, dtype=torch.float32).squeeze()
        
        # Accession number (use PatientID_ScanName as unique identifier)
        acc_no = f"{patient_id}_{scan_name}"
        
        return img_tensor, None, labels, acc_no

    def resize_volume(self, volume, target_shape):
        """
        Resize 3D volume to target shape using trilinear interpolation.
        """
        volume_tensor = torch.tensor(volume).unsqueeze(0).unsqueeze(0)  # Shape: (1, 1, D, H, W)
        resized = F.interpolate(volume_tensor, size=target_shape, mode='trilinear', align_corners=False)
        return resized.squeeze().numpy()

# Evaluation function
def evaluate_model(args, model, dataloader, device):
    model.eval()
    model = model.to(device)
    predictedall = []
    realall = []
    accs = []
    pathologies = ['Medical material', 'Arterial wall calcification', 'Cardiomegaly', 'Pericardial effusion',
                   'Coronary artery wall calcification', 'Hiatal hernia', 'Lymphadenopathy', 'Emphysema',
                   'Atelectasis', 'Lung nodule', 'Lung opacity', 'Pulmonary fibrotic sequela',
                   'Pleural effusion', 'Mosaic attenuation pattern', 'Peribronchial thickening',
                   'Consolidation', 'Bronchiectasis', 'Interlobular septal thickening']

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            inputs, _, labels, acc_no = batch
            labels = labels.float().to(device)
            inputs = inputs.to(device)
            text_tokens = tokenizer("", return_tensors="pt", padding="max_length", 
                                  truncation=True, max_length=200).to(device)
            output = model(False, text_tokens, inputs, device=device)
            predicted = sigmoid(output).cpu().numpy()[0]
            predictedall.append(predicted)
            realall.append(labels.cpu().numpy()[0])
            accs.append(acc_no[0])
            print(f"Accession: {acc_no[0]}", flush=True)

    # Save results
    plotdir = args.save
    os.makedirs(plotdir, exist_ok=True)
    
    with open(f"{plotdir}/accessions.txt", "w") as file:
        for item in accs:
            file.write(f"{item}\n")

    predictedall = np.array(predictedall)
    realall = np.array(realall)
    
    np.savez(f"{plotdir}/labels_weights.npz", data=realall)
    np.savez(f"{plotdir}/predicted_weights.npz", data=predictedall)

    # Generate classification report
    predicted_binary = (predictedall > 0.5).astype(int)
    report = classification_report(realall, predicted_binary, target_names=pathologies, output_dict=True)
    report_df = pd.DataFrame(report).transpose()
    
    # Save AUROC and classification report
    writer = pd.ExcelWriter(f'{plotdir}/aurocs.xlsx', engine='xlsxwriter')
    report_df.to_excel(writer, sheet_name='Classification Report')
    writer.close()

    print("Evaluation complete. Results saved to:", plotdir)

if __name__ == '__main__':
    # Parse arguments
    parser = argparse.ArgumentParser(description="Inference for CT pathology classification")
    parser.add_argument('--data_folder', type=str, 
                        default=r"C:\Users\20203686\OneDrive - TU Eindhoven\TUe\Master\Year 2\MASTER PROJECT\NIFTI PRE CT SCANS",
                        help="Path to preprocessed .npz files")
    parser.add_argument('--reports_file', type=str, 
                        default=r"C:\Users\20203686\OneDrive - TU Eindhoven\TUe\Master\Year 2\MASTER PROJECT\metadata.csv",
                        help="Path to metadata CSV")
    parser.add_argument('--labels', type=str, 
                        default=r"C:\Users\20203686\OneDrive - TU Eindhoven\TUe\Master\Year 2\MASTER PROJECT\labels.csv",
                        help="Path to labels CSV")
    parser.add_argument('--pretrained', type=str, 
                        default=r"C:\Users\20203686\OneDrive - TU Eindhoven\TUe\Master\Year 2\MASTER PROJECT\CT-CLIP\CT_VocabFine_v2.pt",
                        help="Path to pretrained model checkpoint")
    parser.add_argument('--save', type=str, 
                        default=r"C:\Users\20203686\OneDrive - TU Eindhoven\TUe\Master\Year 2\MASTER PROJECT\results",
                        help="Directory to save results")
    args = parser.parse_args()

    # Initialize tokenizer and models
    tokenizer = BertTokenizer.from_pretrained('microsoft/BiomedVLP-CXR-BERT-specialized', do_lower_case=True)
    text_encoder = BertModel.from_pretrained("microsoft/BiomedVLP-CXR-BERT-specialized")
    text_encoder.resize_token_embeddings(len(tokenizer))

    image_encoder = CTViT(
        dim=512,
        codebook_size=8192,
        image_size=480,
        patch_size=20,
        temporal_patch_size=10,
        spatial_depth=4,
        temporal_depth=4,
        dim_head=32,
        heads=8
    )

    clip = CTCLIP(
        image_encoder=image_encoder,
        text_encoder=text_encoder,
        dim_image=294912,
        dim_text=768,
        dim_latent=512,
        extra_latent_projection=False,
        use_mlm=False,
        downsample_image_embeds=False,
        use_all_token_embeds=False
    )

    num_classes = 18
    image_classifier = ImageLatentsClassifier(clip, 512, num_classes)
    
    # Load pretrained weights
    image_classifier.load(args.pretrained)

    # Prepare dataset and dataloader
    ds = CTReportDatasetinfer(data_folder=args.data_folder, csv_file=args.reports_file, labels_csv=args.labels)
    dl = DataLoader(ds, num_workers=8, batch_size=1, shuffle=False)

    # Evaluate model
    evaluate_model(args, image_classifier, dl, torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

ModuleNotFoundError: No module named 'transformer_maskgit'